In [1]:
import pandas as pd
import numpy as np

In [2]:
def file_reader(file_location):
    if(file_location.endswith('csv')):
        return pd.read_csv( file_location , engine = 'python', index_col=0)
    elif (file_location.endswith('tsv')):
        return pd.read_csv( file_location , engine = 'python' ,sep = '\t')

In [3]:
models = [
    'CNN',
    'SVC',
    'GaussianNB',
    'LogisticRegression',
    'RandomForestClassifier'
]

trait_names   = ['cEXT' , 'cNEU' , 'cAGR' , 'cCON' , 'cOPN']

In [9]:
for model in models:
    tweets_preds = file_reader('result/preds_'+model+'.csv')
    tweets  = file_reader('processed_datasets/tweets_processed.csv')
    
    personality_change_dict = {}
    personality_change_stats = {}
    persons = tweets.index

    for trait in trait_names:

        temp_dict = {}
        temp_dict['-1'] = []
        temp_dict['0']  = []
        temp_dict['1']  = []

        for i in range( len(persons) ):
            no = tweets_preds['post_'+trait][i] - tweets_preds['pre_'+trait][i]
            temp_dict[ str(no) ].append( persons[i] )

        temp_dict2 = {}
        temp_dict2['pre_1_to_post_0']  = temp_dict['-1']
        temp_dict2['no_change']         = temp_dict['0']
        temp_dict2['pre_0_to_post_1']  = temp_dict['1']

        temp_dict3 = {}
        temp_dict3['pre_1_to_post_0']  = len(temp_dict['-1'])
        temp_dict3['no_change']         = len(temp_dict['0'])
        temp_dict3['pre_0_to_post_1']  = len(temp_dict['1'])

        personality_change_dict[trait] = temp_dict2 
        personality_change_stats[trait] = temp_dict3
        
    personality_hist_stats = { 0 : 0,1 : 0,2 : 0,3 : 0,4 : 0,5 : 0 }

    for i in tweets_preds.index:
        t_count = 0;
        for trait in trait_names:
            if( tweets_preds['pre_'+trait][i] != tweets_preds['post_'+trait][i] ):
                t_count = t_count + 1;
        personality_hist_stats[t_count] = personality_hist_stats[t_count] + 1
        
    out_result = 'result/'

    import json
    import os
    if( not os.path.isdir(out_result + model) ):
        os.mkdir( out_result + model )
    with open( out_result + model + '/tweets_change_persons.json', 'w') as outfile:
                json.dump(personality_change_dict, outfile)
    with open( out_result + model + '/change_stats.json', 'w') as outfile:
                json.dump(personality_change_stats, outfile)
    with open( out_result + model + '/count_stats.json', 'w') as outfile:
                json.dump(personality_hist_stats, outfile)
    tweets_preds.to_csv( out_result + model + '/tweets_personality_final.csv')

In [ ]:
def plot_graphs(history, trait, str1, str2):
    
    plt.rcParams["figure.figsize"] = (20,8)
    fig,(a,b) =  plt.subplots(1,2)
    
    a.plot( history.history[str1] )
    a.plot( history.history['val_'+str1] )
    a.legend([str1,'val_'+str1])
    plt.title(trait)
    b.plot( history.history[str2] )
    b.plot( history.history['val_'+str2] )
    b.legend([str2,'val_'+str2])

    plt.show() 